<a href="https://colab.research.google.com/github/rbawden/Tutoriel-Normalisation/blob/main/Tutoriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorations sur la normalisation du français moderne

## 1. Setup de l'environnement, téléchargement des fichiers, etc.

Installer les paquets python

In [200]:
!pip install fairseq@git+git://github.com/pytorch/fairseq.git@5a75b079bf8911a327940c28794608e003a9fa52 
!pip install sentencepiece sacrebleu hydra-core omegaconf==2.0.5 gdown==4.2.0 
import warnings
warnings.filterwarnings('ignore')

  Using cached fairseq-1.0.0a0+5a75b07-cp37-cp37m-macosx_10_9_x86_64.whl


Télécharger les données et les modèles depuis Google Drive et structures-les dans les dossiers `data/`, `models/`

In [121]:
!bash download_files.sh

## 2. Préparation des données à normaliser

Fonctions pour lire le contenu d'un fichier ligne par ligne et pour les lire depuis un fichier

In [122]:
# lire un fichier ligne par ligne
def read_file(filename):
  list_sents = []
  with open(filename) as fp:
    for line in fp:
      list_sents.append(line.strip())
  return list_sents

# écrire une liste de phrases dans un fichier
def write_file(list_sents, filename):
    with open(filename, 'w') as fp:
        for sent in list_sents:
            fp.write(sent + '\n')

Lire le contenu du texte source à normaliser (`dev.src`) et le texte cible 'de référence', c'est-à-dire le texte correctement normalisé (`dev.trg`)

In [123]:
data_src = read_file('data/dev.src')
data_trg = read_file('data/dev.trg')

Visualiser le début des textes sources (src) et cibles (trg)

In [124]:
for i in range(4):
    print('src = ', data_src[i])
    print('trg = ', data_trg[i])
    print('--')

src =  1.
trg =  1.
--
src =  1. QVe cette propoſtion, qu'vn eſpace eſt vuidé, repugne au ſens commun.
trg =  1. QUe cette proposition, qu'un espace est vidé, répugne au sens commun.
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre du vuide dans leur interualle;
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre du vide dans leur intervalle;
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre ce vide apparent dans leur intervalle:
--


Charger le modèle de segmentation en sous-mots (`bpe_joint_1000.model`)

In [125]:
import sentencepiece
spm = sentencepiece.SentencePieceProcessor(model_file='data/bpe_joint_1000.model')

Appliquer le modèle de segmentation sur les données à normaliser

In [130]:
data_src_sp = spm.encode(data_src, out_type=str)

Écrire le texte pre-traité dans un fichier `dev.sp.src`

In [131]:
write_file([' '.join(phrase) for phrase in data_src_sp], 'data/dev.sp.src')

Visualiser le début du fichier

In [132]:
data_src_sp[:2]

[['▁1', '.'],
 ['▁1',
  '.',
  '▁Q',
  'V',
  'e',
  '▁cette',
  '▁prop',
  'ost',
  'ion',
  ',',
  '▁qu',
  "'",
  'vn',
  '▁esp',
  'ace',
  '▁est',
  '▁v',
  'ui',
  'd',
  'é',
  ',',
  '▁re',
  'p',
  'u',
  'gne',
  '▁au',
  '▁sens',
  '▁comm',
  'un',
  '.']]

Définir une fonction pour détokeniser (pour plus tard)

In [133]:
def decode_sp(list_sents):
    return [''.join(sent).replace(' ', '').replace('▁', ' ') for sent in list_sents]

Visualiser à quoi ressemble le texte détokenisé (Spoiler: il devrait ressembler au texte de départ)

In [134]:
decode_sp(data_src_sp[:5])

[' 1.',
 " 1. QVe cette propostion, qu'vn espace est vuidé, repugne au sens commun.",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre du vuide dans leur interualle;",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:",
 ' 2.']

## 3. Appliquer le modèle de normalisation

Appliquer le modèle de normalisation sur le début des données pre-traitées (ça prend moins de temps pour tester que normaliser tout le texte)

Il y aura un message "UserWarning", mais vous pouvez l'ignorer - ce n'est pas grave.

Explications:
- head -n 10 affiche les 10 premières phrases
- ces 10 premières lignes sont donné à fairseq-interactive
- le résultat va dans `data/dev.sp.norm.trg.10.output`

In [204]:
!head -n 10 data/dev.sp.src | fairseq-interactive models/norm/ --source-lang src --target-lang trg --path models/norm/lstm_norm.pt > data/dev.sp.norm.trg.10.output

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/sequence_generator.py:669: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // b

La sortie de fairseq-interactive donne quelque chose comme ceci:

```
S-0     ▁1 .
H-0     -0.00011481383990030736 ▁1 .
P-0     -0.0000 -0.0003 -0.0000
S-1     ▁1 . ▁Q V e ▁cette ▁prop ost ion , ▁qu ' vn ▁esp ace ▁est ▁v ui d é , ▁re p u gne ▁au ▁sens ▁comm un .
H-1     -0.039981111884117126   ▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .
P-1     -0.0000 -0.0000 -0.0043 -0.0632 -0.0006 -0.0000 -0.0001 -0.9353 -0.0001 -0.0012 -0.0000 0.0000 -0.0001 -0.0078 -0.0070 -0.0000 -0.0022 -0.1168 -0.0001 -0.0000 -0.0000 -0.0389 -0.0157 -0.0053 -0.0000 -0.0000 -0.0001 -0.0000 -0.0004 -0.0000
S-2     ▁1 . ▁Q V e ▁tous ▁les ▁cor p s ▁ont ▁re p u gn ance ▁à ▁se ▁se p are r ▁l ' vn ▁de ▁l ' autre , ▁& ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter u al le ;
W-2     0.682   seconds
H-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
D-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
P-2     -0.0000 -0.0001 -0.0040 -0.1684 -0.0004 -0.0000 -0.0000 -0.0000 -0.0007 -0.0000 -0.0001 -0.1220 -0.0063 -0.0002 -0.0137 -0.0000 -0.0000 -0.0002 -0.0001 -0.0248 -0.0022 -0.0003 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0002 -0.0001 -0.0000 -0.0000 -0.0000 -0.0000 -0.0383 -0.0173 -0.0006 -0.0000 -0.0000 -0.0000 -0.0066 -0.0016 -0.4856 -0.0007 -0.0002 -0.0000
```

Les informations intéressantes pour l'exemple `i`:

- S-i: le texte source
- H-i: le score de l'hypothèse et l'hypothèse du modèle (c'est-à-dire la prédiction)
- P-i: les scores de chaque sous-token produit par le modèle

Fonction pour extraire l'hypothèse de ce fichier

In [205]:
def extract_hypothesis(filename):
    outputs = []
    with open(filename) as fp:
        for line in fp:
            # seulement les lignes qui commencet par H- (pour Hypothèse)
            if 'H-' in line:
                # prendre la 3ème colonne (c'est-à-dire l'indice 2)
                outputs.append(line.strip().split('\t')[2])
    return outputs

Extraire les hypothèses du fichier produit

In [206]:
outputs = extract_hypothesis('data/dev.sp.norm.trg.10.output')
outputs[:3]

['▁1 .',
 "▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .",
 "▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;"]

Post-traiter le texte avec la fonction précedemment définie (dé-segmenter)

In [207]:
outputs_postproc = decode_sp(outputs)
outputs[:3]

['▁1 .',
 "▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .",
 "▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;"]

Écrire le résultat dans un fichier

In [208]:
write_file(outputs_postproc, 'data/dev.sp.norm.10.trg')

## 4. Évaluation du résultat

- BLEU: le métrique d'évaluation le plus fréquemment utilisé en traduction automatique
- ChrF: CharacterF score (like BLEU but based on n-grams of characters)
- TER: translation edit rate

In [141]:
from sacrebleu.metrics import BLEU, CHRF, TER
bleu = BLEU()
bleu.corpus_score(outputs_postproc, [data_trg[:10]])

BLEU = 85.90 94.5/88.4/83.2/78.4 (BP = 1.000 ratio = 1.000 hyp_len = 199 ref_len = 199)

In [142]:
chrf = CHRF()
chrf.corpus_score(outputs_postproc, [data_trg[:10]])

chrF2 = 95.73

In [143]:
ter = TER()
ter.corpus_score(outputs_postproc, [data_trg[:10]])

TER = 6.55

Une évaluation plus adaptée : la précision au niveau de chaque mot

In [163]:
import align
import importlib
importlib.reload(align)

<module 'align' from '/Users/rbawden/Research/tutorials/Tutoriel-Normalisation/align.py'>

In [164]:
alignments = align.align('data/dev.sp.src', 'data/dev.sp.norm.trg')

In [165]:
num_diff = 0
total = 0
for sentence in alignments:
    for word in sentence:
        if '>' in word:
            num_diff += 1
        total += 1
print('Accuracy = ' + str(num_diff/total))

Accuracy = 0.8638743455497382


# 5. Tester le modèle de dénormalisation

### Preparer les données normalisées qui vont être dénormalisées

In [211]:
data_trg_sp = spm.encode(data_trg, out_type=str) # segmenter le texte en sous-mots
decade_token = '▁<decade=162> '
write_file([' '.join([decade_token] + phrase) for phrase in data_trg_sp], 'data/dev.sp.trg')

### Normaliser le texte

In [212]:
!head -n 10 data/dev.sp.trg | fairseq-interactive models/denorm --source-lang trg --target-lang src --path models/denorm/lstm_denorm.pt > data/dev.sp.denorm.src.10.output

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/sequence_generator.py:669: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // b

### Post-traiter la sortie du modèle

In [213]:
outputs = extract_hypothesis('data/dev.sp.denorm.src.10.output')
outputs_postproc = decode_sp(outputs)
write_file(outputs_postproc, 'data/dev.sp.denorm.10.src')
outputs_postproc[:3]

[' 1.',
 " 1. QVe cette proposition, qu'vn espace est vidé, repugne au sens commun.",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre du vide dans leur interualle;"]

In [226]:
def denormalise(sent, decade):
    assert int(decade) >=1600 and int(decade) < 1700, 'Your decade must be between 1600 and 1690'
    # generate temporary file
    filetmp = 'data/tmp_denorm.sp.trg.tmp'
    # preprocessing
    input_sp = spm.encode(sent, out_type=str)
    # add decade token to each sentence
    decade_token = '▁<decade=' + str(decade)[:3] + '>'
    input_sp_sents = [' '.join([decade_token] + input_sp)]
    write_file(input_sp_sents, filetmp)
    print(input_sp_sents)
    # denormalisation
    !cat data/tmp_denorm.sp.trg.tmp | fairseq-interactive models/denorm --source-lang trg --target-lang src --path models/denorm/lstm_denorm.pt > data/tmp_denorm.sp.trg.tmp.output 2> /tmp/dev
    # postprocessing
    outputs = extract_hypothesis('data/tmp_denorm.sp.trg.tmp.output')
    outputs_postproc = decode_sp(outputs)
    return outputs_postproc[0]

In [227]:
print(denormalise("Il est beau.", 1620))

['▁<decade=162> ▁Il ▁est ▁beau .']
 Il est beau.


In [228]:
def normalise(sent):
    # generate temporary file
    filetmp = 'data/tmp_norm.sp.src.tmp'
    # preprocessing
    input_sp = spm.encode(sent, out_type=str)
    # add decade token to each sentence
    input_sp_sents = [' '.join(input_sp)]
    write_file(input_sp_sents, filetmp)
    print("preprocessed = ", input_sp_sents)
    # denormalisation
    !cat data/tmp_norm.sp.src.tmp | fairseq-interactive models/norm --source-lang src --target-lang trg --path models/norm/lstm_norm.pt > data/tmp_norm.sp.src.tmp.output 2> /tmp/dev
    # postprocessing
    outputs = extract_hypothesis('data/tmp_norm.sp.src.tmp.output')
    outputs_postproc = decode_sp([outputs])
    return outputs_postproc

In [ ]:
normalise("1. QVe cette propostion, qu'vn espace est vuidé, repugne au sens commun.")

preprocessed =  ["▁1 . ▁Q V e ▁cette ▁prop ost ion , ▁qu ' vn ▁esp ace ▁est ▁v ui d é , ▁re p u gne ▁au ▁sens ▁comm un ."]


# 6. Quelques extensions

### Appliquer le modèle sur le texte entier (ou un plus grand nombre de phrases si vous ne voulez pas attendre).
Le fichier entièrement normalisé est aussi disponible ici: `data/dev.norm.full.trg` si vous voulez juste l'évaluer et l'analyser

In [80]:
!cat data/dev.sp.src | fairseq-interactive models/ --source-lang src --target-lang trg --path models/lstm_norm.pt > data/dev.sp.norm.trg.output

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/sequence_generator.py:669: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // b

### Faire une analyse qualitative de résultats.

In [ ]:
def print_most_frequent_diffs(alignments, show_n=10):
    # TODO
    return

print_most_frequent_errors(alignments)

### Faire une analyse qualitative des différences entre un texte source et cible - faire cette analyse sur le jeu d'entraînement

### Créer une baseline par règles. Comme ressources supplémentaires, vous avez un lexique de mots en français contemporain et quelques fonctions

In [40]:
# read a lexicon in the mlex format
import gzip
def read_lexicon(filename):
    words = []
    with gzip.open(filename, 'rt') as fp:
        for line in fp:
            words.append(line.split('\t')[0])
    return words


def similarity_levenhstein(word1, word2):
    dist, matrix, backpointers = levenshtein('@' + word1, '@' + word2)
    return dist


def similarity_common_chars(word1, word2):
    # TODO
    return

In [41]:
lexicon = read_lexicon('data/lefff-3.4.mlex.gz')

In [46]:
print('Le lexique contient ' + str(len(lexicon)) + ' entrées.')

Le lexique contient 549274 entrées.


In [55]:
def normalise_sent(sent, normalise_word_function):
    norm_sent = []
    for word in sent.split():
        norm_sent.append(normalise_word_function(word))
    return ' '.join(norm_sent)

In [99]:
# fonction qui renvoie simplement le même mot
def return_word(word):
    return word

In [100]:
normalise_word_function = return_word
normalise_sent("QVe cette propoſtion, qu'vn eſpace eſt vuidé", normalise_word_function)

"QVe cette propoſtion, qu'vn eſpace eſt vuidé"

In [101]:
def replace_long_s(word):
    word = word.replace('ſ', 's')
    return word

In [102]:
normalise_word_function = replace_long_s
normalise_sent("QVe cette propoſtion, qu'vn eſpace eſt vuidé", normalise_word_function)

"QVe cette propostion, qu'vn espace est vuidé"

Vous pouvez créer une fonction qui contient plus de remplacement (comme `word.replace(before, after)`).

Parfois, un remplacement peut être contextuel, donc si vous connaissez les expressions régulières vous pouvez les utiliser aussi.


In [107]:
import re
def replace_regex(word):
    word = word.replace('ſ', 's')
    word = re.sub("([Qq])v", r'\1u', word)
    word = re.sub("([Qq])V", r'\1U', word)
    word = re.sub("([' ])vn(e?)", r'\1un\2', word)
    return word

normalise_word_function = replace_regex
normalise_sent("QVe cette propoſtion, qu'vn eſpace eſt vuidé", normalise_word_function)

"QUe cette propostion, qu'un espace est vuidé"